# QE input script generator
This will automate the process of writing input scripts for Quantum Espresso to do the structural optimization of functionalized mofs using the pw.x (plane wave) package and pseudopotential files. 

**NOTE:** many default arguements will be preset.

In [1]:
cd("..")

In [2]:
using PorousMaterials
using Printf

┌ Info: Precompiling PorousMaterials [68953c7c-a3c7-538e-83d3-73516288599e]
└ @ Base loading.jl:1273
  ** incremental compilation may be fatally broken for this module **

┌ Warning: Package PorousMaterials does not have PyCall in its dependencies:
│ - If you have PorousMaterials checked out for development and have
│   added PyCall as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with PorousMaterials
└ Loading PyCall into PorousMaterials from project dependency, future warnings for PorousMaterials are suppressed.


In [3]:
PorousMaterials.PATH_TO_DATA

"/home/ng/DTRA/data"

In [4]:
cd("structural_relaxation")

**TODO:**
* fix file output location
* loop over all functionalized mofs

In [5]:
atomic_masses = read_atomic_masses()

Dict{Symbol,Float64} with 100 entries:
  :Xe__    => 131.293
  :Cl      => 35.453
  :Al      => 26.9815
  :Be      => 9.01218
  :Re      => 186.207
  :O_RCOO  => 15.9994
  :Cr      => 51.996
  :Na      => 22.9898
  :N_in_N2 => 14.0067
  :VOID    => 0.0
  :Sb      => 121.76
  :Kr      => 83.798
  :Ni      => 58.6934
  :S       => 32.065
  :CH4     => 16.04
  :O_zeo   => 15.9994
  :Ru      => 101.07
  :Tm      => 168.934
  :C_sp3   => 12.0107
  :Xe_     => 131.293
  :Nd      => 144.242
  :O       => 15.9994
  :Tb      => 158.925
  :Th      => 232.038
  :Zr      => 91.224
  ⋮        => ⋮

In [6]:
# which_crystal = "NiPyC2"
# replication_id = "sc211_" # super cell 2x1x1
# qe_run_type = "relax"

# # name of the directory containing functionalixed structures
# mof_dir_name = which_crystal * "_" * replication_id * qe_run_type

# @eval PorousMaterials PATH_TO_CRYSTALS = joinpath(pwd(), "mof_construction",
#     "NiPyC2_sc211_relax") 

In [7]:
# # The name of the fragment
# # file must be a .xyz file located in ./fragments/
# # The atom species that is bonded to the :C_aro_R atom on the ring segment of the Fragment
# fragment_list = ["Br", "CH2-CH2-CH3", "CH2-CH3", "CH3", 
#                 "Cl", "C-N", "F", "NH2", "N-NH", 
#                 "O-CH2-CH2-CH3", "O-CH2-CH3", "O-CH3", "OH"]

# # The type of Arene Substitution ("ortho", "meta", "all") 
# # TODO: "all" not yet implimented
# substitution_types = ["ortho", "meta"]

In [8]:
# base_name = "NiPyC2_relax"
# xtals = Dict{String, Crystal}()

# for frag in fragment_list
#     for sub_typ in substitution_types
#         xtal_name = base_name * "_" * sub_typ * "_functionalized_" * frag
#         xtals[xtal_name] = Crystal(xtal_name * ".cif")
#     end
# end

In [9]:
# import the functionalized mofs 
# crystal = Crystal("NiPyC2_relax_meta_functionalized_OH.cif")
crystal = Crystal("NiPyC2.cif")

strip_numbers_from_atom_labels!(crystal)
# relabel Ni atoms
# @warn "we labeled Ni's 1-4, but not sure if we need to relabel as Ni1, Ni2 and have two of each."
# ni_count = 0
# for (i, atom) in enumerate(crystal.atoms.species)
#     if atom == :Ni
#         ni_count += 1
#         cnt = mod(ni_count, 2) + 1
#         crystal.atoms.species[i] = Symbol(@sprintf("Ni%d", cnt))
#     end
# end

┌ Info: Crystal NiPyC2.cif has Pn space group. I am converting it to P1 symmetry.
│         To afrain from this, pass `convert_to_p1=false` to the `Crystal` constructor.
└ @ PorousMaterials /home/ng/.julia/dev/PorousMaterials/src/crystal.jl:405


In [10]:
crystal.box.f_to_c


3×3 Array{Float64,2}:
 6.2528   7.66837e-16  -0.227594   
 0.0     12.5234        6.43209e-16
 0.0      0.0          10.2743     

## paths to pseudo-potential libraries

populate dictionary with names of pseudopotential files

In [11]:
functional_to_atom_to_pp_file = Dict{String, Dict{Symbol, String}}()

functional_to_atom_to_pp_file["pbesol"] = Dict{Symbol, String}()

functional_to_atom_to_pp_file["pbesol"][:Ni] = "Ni.pbesol-n-nc.UPF"
functional_to_atom_to_pp_file["pbesol"][:O] = "O.pbesol-nc.UPF"
functional_to_atom_to_pp_file["pbesol"][:C] = "C.pbesol-nc.UPF"
functional_to_atom_to_pp_file["pbesol"][:N] = "N.pbesol-nc.UPF"
functional_to_atom_to_pp_file["pbesol"][:H] = "H.pbesol-n-nc.UPF"
functional_to_atom_to_pp_file["pbesol"][:F] = "F.pbesol-n-nc.UPF"
functional_to_atom_to_pp_file["pbesol"][:Cl] = "Cl.pbesol-n-nc.UPF"
functional_to_atom_to_pp_file["pbesol"][:Br] = "Br.pbesol-n-nc.UPF"
functional_to_atom_to_pp_file["pbesol"][:P] = "P.pbesol-n-nc.UPF"
functional_to_atom_to_pp_file["pbesol"][:S] = "S.pbesol-n-nc.UPF"

# functional_to_atom_to_pp_file["VDW"] = Dict{Symbol, String}()


"S.pbesol-n-nc.UPF"

## default QE structural relaxation params

In [12]:
"""
    input_params = qe_input_params(crystal; kwargs)

create a dictionary consisting of input arguments for QE structural relaxation
needed for the script and set default values
then populate with values specific to each material
link with full list and description of input params
https://www.quantum-espresso.org/Doc/INPUT_PW.html#__top__

WARNING: tailored to Nickel
"""
function qe_structural_opt_input_params(crystal::Crystal; kwargs...)
    @warn "tailored to NiPyC and their analogues"
    
    input_params = Dict{Symbol, Any}()
    # &control params
    input_params[:calculation] = "relax" 
    input_params[:prefix] = split(crystal.name, ".")[1]
    input_params[:pseudo_dir] = "./pseudo/" #joinpath(pwd(), "structural_relaxation", "pseudo")
    input_params[:outdir] = "./" #joinpath(pwd(), "structural_relaxation")
    input_params[:nstep] = 200

    # &SYSTEM params
    input_params[:ibrav] = 0
    input_params[:nat] = crystal.atoms.n
    input_params[:ntyp] = length(unique(crystal.atoms.species))
    input_params[:tot_charge] = 0.0
    input_params[:ecutwfc] = 100.0
    input_params[:vdw_corr] = "grimme-d2"
    input_params[:occupations] = "smearing"
    input_params[:degauss] = 0.2
    input_params[:smearing] = "mv"
    input_params[:nspin] = 2
    input_params[:sub_lat_species_index] = ["1", "2"]
    input_params[:starting_magnetization] = [0.25, 0.25]
    # tot_mag = total number of unpaired electrons in sys 
    input_params[:tot_magnetization] = 
        sum(crystal.atoms.species .== :Ni) * 2

    # &ELECTRON params
    input_params[:electron_maxstep] = 90
    input_params[:mixing_mode] = "local-TF"
    input_params[:diagonalization] = "david"

    # &IONS params
    input_params[:ion_dynamics] = "bfgs"

    # &CELL params
    input_params[:cell_dynamics] = "bfgs"
    input_params[:cell_dofree] = "xyz"

    # K_POINTS params
    input_params[:k_points] = [String("{gamma}")]
    
    # over-ride default params with ones passed in kwargs
    for (key, value) in kwargs
        input_params[key] = value
    end
    return input_params
end

input_params = qe_structural_opt_input_params(crystal)

┌ Warning: tailored to NiPyC and their analogues
└ @ Main In[12]:13


Dict{Symbol,Any} with 25 entries:
  :starting_magnetization => [0.25, 0.25]
  :ecutwfc                => 100.0
  :vdw_corr               => "grimme-d2"
  :occupations            => "smearing"
  :diagonalization        => "david"
  :prefix                 => "NiPyC2"
  :nspin                  => 2
  :outdir                 => "./"
  :pseudo_dir             => "./pseudo/"
  :ntyp                   => 5
  :smearing               => "mv"
  :ibrav                  => 0
  :degauss                => 0.2
  :mixing_mode            => "local-TF"
  :k_points               => ["{gamma}"]
  :sub_lat_species_index  => ["1", "2"]
  :tot_charge             => 0.0
  :calculation            => "relax"
  :ion_dynamics           => "bfgs"
  :cell_dynamics          => "bfgs"
  :nstep                  => 200
  :nat                    => 54
  :cell_dofree            => "xyz"
  :electron_maxstep       => 90
  :tot_magnetization      => 4

In [13]:
function check_psuedopotential_coverage(crystal::Crystal, functional::String)
    for atom in unique(crystal.atoms.species)
        if ! (atom in keys(functional_to_atom_to_pp_file[functional]))
            error(@sprintf("atom %s not in `atom_to_psuedopotential_file`", atom))
        end
    end
end

check_psuedopotential_coverage(crystal, "pbesol")

In [14]:
function generate_qe_structural_opt_input_file(crystal::Crystal, input_params::Dict{Symbol, Any}, functional::String)
    # check pseudo-potential coverage
    check_psuedopotential_coverage(crystal)
    
    qe_input_filename = "pw." * split(crystal.name, ".cif")[1] *
        "_" * functional * "_" * input_params[:calculation] * ".in"
    # make this folder if it doesn't exist
    if ! isdir("QE_input_scripts")
        mkdir("QE_input_scripts")
    end
    qe_input_file = open(joinpath("QE_input_scripts", qe_input_filename)
        , "w")

    # write &Control
    @printf(qe_input_file,
        """
        &CONTROL
           calculation = '%s',
           prefix = '%s',
           pseudo_dir = '%s',
           outdir = '%s'
           nstep = %.0f,
        /
        """,
        input_params[:calculation], input_params[:prefix],
        input_params[:pseudo_dir],
        input_params[:outdir], input_params[:nstep]
    )

    # write &SYSTEM
    @printf(qe_input_file,
        """
        &SYSTEM
           ibrav = %.0f,
           nat = %.0f,
           ntyp = %0.f,
           tot_charge = %f,
           ecutwfc = %f,
           vdw_corr = '%s',
           occupations = '%s',
           degauss = %f,
           smearing = '%s',
           nspin = %.0f ,
           starting_magnetization(%s) = %f,
           ! starting_magnetization(%s) = %f
           tot_magnetization = %f
        /
        """, 
        input_params[:ibrav], input_params[:nat],
        input_params[:ntyp], input_params[:tot_charge],
        input_params[:ecutwfc], input_params[:vdw_corr],
        input_params[:occupations], input_params[:degauss],
        input_params[:smearing], input_params[:nspin],
        input_params[:sub_lat_species_index][1],
        input_params[:starting_magnetization][1],
        input_params[:sub_lat_species_index][2],
        input_params[:starting_magnetization][2],
        input_params[:tot_magnetization]
    )

    # write &ELECTRONS
    @printf(qe_input_file,
        """
        &ELECTRONS
        electron_maxstep = %0.f,
        mixing_mode = '%s',
        diagonalization = '%s',
        /
        """,
        input_params[:electron_maxstep], input_params[:mixing_mode], 
        input_params[:diagonalization]
    )

    # write &IONS
    @printf(qe_input_file,
        """
        &IONS
        ion_dynamics = '%s',
        /
        """,
        input_params[:ion_dynamics]
    )


    # write &CELL
    @printf(qe_input_file,
        """
        &CELL
        cell_dynamics = '%s'
        cell_dofree = '%s'
        /
        """,
        input_params[:cell_dynamics], input_params[:cell_dofree]
    )

    # write ATOMIC_SPECIES
    @printf(qe_input_file,
        """
        ATOMIC_SPECIES
        """
    )
    
    @warn "address Ni vs Ni2"
    for atom in unique(crystal.atoms.species)
        @printf(qe_input_file,
            """
            %s    %f    %s
            """,
            atom, atomic_masses[atom], functional_to_atom_to_pp_file[functional][atom]
        )
    end

    # write K_POINTS
    @printf(qe_input_file,
        """
        K_POINTS %s
        """,
        input_params[:k_points][1]
    )


    # write CELL_PARAMETERS
    @printf(qe_input_file,
        """
        CELL_PARAMETERS {angstrom}
        """
    )
    for d = 1:3
        @printf(qe_input_file,
            "     %f   %f   %f\n",
            crystal.box.f_to_c[1, d], crystal.box.f_to_c[2, d], crystal.box.f_to_c[3, d]
        )
    end

    # write ATOMIC_POSITIONS
    #symbol #x #y #z
    @printf(qe_input_file, "ATOMIC_POSITIONS {crystal}\n")


    for a = 1:crystal.atoms.n
        @printf(qe_input_file, 
            "%s    %f  %f  %f\n",
            crystal.atoms.species[a], 
            crystal.atoms.coords.xf[1, a], 
            crystal.atoms.coords.xf[2, a], 
            crystal.atoms.coords.xf[3, a]
        )
    end
    close(qe_input_file)
    return qe_input_filename
end

generate_qe_structural_opt_input_file (generic function with 1 method)

In [15]:
input_params = qe_structural_opt_input_params(crystal);

┌ Warning: tailored to NiPyC and their analogues
└ @ Main In[12]:13


In [16]:
qe_input_filename = generate_qe_structural_opt_input_file(crystal, input_params, "pbesol")

MethodError: MethodError: no method matching check_psuedopotential_coverage(::Crystal)
Closest candidates are:
  check_psuedopotential_coverage(::Crystal, !Matched::String) at In[13]:2

In [17]:
sum(crystal.atoms.species .== :Ni)

2

In [18]:
run(`cat QE_input_scripts/$qe_input_filename`)

UndefVarError: UndefVarError: qe_input_filename not defined